In [1]:
import tensorflow as tf
from tensorflow import keras

In [19]:
def build_model(input_shape=(32,32,1), num_classes=10):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), padding='valid', activation='tanh', input_shape=input_shape),
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2)),

        tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), strides=1, padding='valid', activation='tanh'),
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=120, activation='tanh'),

        tf.keras.layers.Dense(units=84, activation='tanh'),

        tf.keras.layers.Dense(units=num_classes, activation='softmax')
    ])

    return model

In [21]:
lenet5 = build_model()
lenet5.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_14 (Avera  (None, 14, 14, 6)        0         
 gePooling2D)                                                    
                                                                 
 conv2d_22 (Conv2D)          (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_15 (Avera  (None, 5, 5, 16)         0         
 gePooling2D)                                                    
                                                                 
 flatten_9 (Flatten)         (None, 400)               0         
                                                                 
 dense_31 (Dense)            (None, 120)              

In [22]:
def identity_block(x, filters, kernel_size=3):
    """Identity Block của ResNet"""
    f1, f2, f3 = filters

    x_shortcut = x

    # Tầng Convolutional đầu tiên
    x = tf.keras.layers.Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Tầng Convolutional thứ hai
    x = tf.keras.layers.Conv2D(f2, kernel_size=(kernel_size, kernel_size), strides=(1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Tầng Convolutional thứ ba
    x = tf.keras.layers.Conv2D(f3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # Kết hợp với đầu vào
    x = tf.keras.layers.Add()([x, x_shortcut])
    x = tf.keras.layers.Activation('relu')(x)

    return x

def convolutional_block(x, filters, kernel_size=3, strides=2):
    """Convolutional Block của ResNet"""
    f1, f2, f3 = filters

    x_shortcut = x

    # Tầng Convolutional đầu tiên
    x = tf.keras.layers.Conv2D(f1, kernel_size=(1, 1), strides=(strides, strides), padding='valid')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Tầng Convolutional thứ hai
    x = tf.keras.layers.Conv2D(f2, kernel_size=(kernel_size, kernel_size), strides=(1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Tầng Convolutional thứ ba
    x = tf.keras.layers.Conv2D(f3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # Tầng Shortcut
    x_shortcut = tf.keras.layers.Conv2D(f3, kernel_size=(1, 1), strides=(strides, strides), padding='valid')(x_shortcut)
    x_shortcut = tf.keras.layers.BatchNormalization()(x_shortcut)

    # Kết hợp với đầu vào
    x = tf.keras.layers.Add()([x, x_shortcut])
    x = tf.keras.layers.Activation('relu')(x)

    return x

def ResNet50(input_shape=(224, 224, 3), num_classes=1000):
    """Xây dựng mô hình ResNet-50"""

    input_tensor = tf.keras.layers.Input(shape=input_shape)

    # Tầng đầu tiên
    x = tf.keras.layers.ZeroPadding2D(padding=(3, 3))(input_tensor)
    x = tf.keras.layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='valid')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)

    # Các Blocks
    x = convolutional_block(x, filters=[64, 64, 256], strides=1)
    x = identity_block(x, filters=[64, 64, 256])
    x = identity_block(x, filters=[64, 64, 256])

    x = convolutional_block(x, filters=[128, 128, 512])
    x = identity_block(x, filters=[128, 128, 512])
    x = identity_block(x, filters=[128, 128, 512])
    x = identity_block(x, filters=[128, 128, 512])

    x = convolutional_block(x, filters=[256, 256, 1024])
    x = identity_block(x, filters=[256, 256, 1024])
    x = identity_block(x, filters=[256, 256, 1024])
    x = identity_block(x, filters=[256, 256, 1024])
    x = identity_block(x, filters=[256, 256, 1024])

    x = convolutional_block(x, filters=[512, 512, 2048])
    x = identity_block(x, filters=[512, 512, 2048])
    x = identity_block(x, filters=[512, 512, 2048])

    # Tầng Global Average Pooling
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    # Tầng Fully Connected
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=input_tensor, outputs=x)

    return model

# Tạo mô hình
resnet50_model = ResNet50()
resnet50_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_2[0][0]']                
                                                                                                  
 conv2d_23 (Conv2D)             (None, 112, 112, 64  9472        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                            